In [1]:
import pandas as pd
import numpy as np
import itertools

# funciones desarrolladas
from functions.cargar_data import *

In [2]:
# calcula migrantes internos entre localidades
pmigr = cargar_migrantes_internos()

mig_loc  = pmigr.groupby(['loc_origen','loc_destino']).size().to_frame('personas_mig').reset_index()
mig_loc['cod'] = (mig_loc.loc_origen.astype(str) + mig_loc.loc_destino.astype(str)).astype(int)

print(mig_loc.shape)
mig_loc.head(2)

(12323, 4)


,loc_origen,loc_destino,personas_mig,cod
0,1020,2220,635,10202220
1,1020,2521,90,10202521


In [3]:
# levanta población por localidad
poblacion = pd.read_csv('tablas/localidades_censales_2011.csv', usecols=['codloc','poblacion'])

print(poblacion.shape)
poblacion.head(2)

(615, 2)


,codloc,poblacion
0,1020,1304729
1,2220,40657


In [4]:
# levanta distancias
dist = pd.read_csv('tablas/df_distancias_localidades.csv', usecols=['cod','distancia_m'])
print(dist.shape)
dist.head(2)

(377610, 2)


,cod,distancia_m
0,22202521,137927
1,22202522,111408


In [5]:
def permutations(array):
    "Genera las combinaciones de pares de códigos posibles devolviendo un DF"
    _combi = list(itertools.permutations(array, 2))
    _cod_ori = [x[0] for x in _combi]
    _cod_des = [x[1] for x in _combi]
    _combi_codes = [int(str(x[0]) + str(x[1])) for x in _combi]
    data_tuples = list(zip(_combi_codes, _cod_ori, _cod_des, ))
    cod = pd.DataFrame(data_tuples, columns=['cod', 'cod_ori', 'cod_des'])
    return cod

In [6]:
cod = permutations(poblacion.codloc.values)
cod.head(2)

,cod,cod_ori,cod_des
0,10202220,1020,2220
1,10202521,1020,2521


In [7]:
# pega población en origen y destino
dd = cod.merge(poblacion, left_on='cod_ori', right_on='codloc', how='left')
dd = dd.merge(poblacion, left_on='cod_des', right_on='codloc', how='left')
dd.drop(['codloc_x', 'codloc_y'], axis=1, inplace=True)
dd.rename({'poblacion_x': 'poblacion_ori', 'poblacion_y': 'poblacion_des'}, inplace=True, axis=1)
print(dd.shape)
dd.head()

(377610, 5)


,cod,cod_ori,cod_des,poblacion_ori,poblacion_des
0,10202220,1020,2220,1304729,40657
1,10202521,1020,2521,1304729,12200
2,10202522,1020,2522,1304729,2659
3,10202621,1020,2621,1304729,2531
4,10202721,1020,2721,1304729,380


In [8]:
# pega población migrante por díada
dd = dd.merge(mig_loc, on='cod', how='left').drop(['loc_origen', 'loc_destino'], axis=1)
dd.head()

,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig
0,10202220,1020,2220,1304729,40657,635.0
1,10202521,1020,2521,1304729,12200,90.0
2,10202522,1020,2522,1304729,2659,17.0
3,10202621,1020,2621,1304729,2531,9.0
4,10202721,1020,2721,1304729,380,NaN


In [9]:
# pega distancias
dd = dd.merge(dist, on='cod')

In [10]:
dd.head()

,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m
0,10202220,1020,2220,1304729,40657,635.0,583715
1,10202521,1020,2521,1304729,12200,90.0,612802
2,10202522,1020,2522,1304729,2659,17.0,617076
3,10202621,1020,2621,1304729,2531,9.0,595785
4,10202721,1020,2721,1304729,380,NaN,628213


In [11]:
# exporta a CSV
dd.to_csv('tablas/dd_localidades.csv', index=False)